In [1]:
import torch
import numpy as np
# import open3d as o3d
from pointnet.model import PointNetCls
import os
from __future__ import print_function
from show3d_balls import showpoints
import argparse
import numpy as np
import torch
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
from pointnet.model import PointNetDenseCls
import matplotlib.pyplot as plt
import yaml
import os

Shared library loaded successfully


# 一. 加载数据

In [3]:
def load_off(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # if lines[0].strip() != 'OFF':
        #     raise ValueError('Not a valid OFF header')

        parts = lines[1].strip().split()
        num_vertices = int(parts[0])

        vertices = []
        for i in range(2, 2 + num_vertices):
            vertex = list(map(float, lines[i].strip().split()))
            vertices.append(vertex)

        return np.array(vertices)

# 二. 预测

In [5]:
def load_off_file(filename):
    """读取OFF文件并提取点云数据"""
    with open(filename, 'r') as file:
        lines = file.readlines()
        if lines[0].strip() != 'OFF':
            raise ValueError('Not a valid OFF file')

        n_verts, n_faces, _ = map(int, lines[1].strip().split())
        verts = [list(map(float, line.strip().split())) for line in lines[2:2 + n_verts]]

        return np.array(verts, dtype=np.float32)

def segment_point_cloud(points_arr, segment_model_path): 

    # 读取OFF文件，提取点云数据
    point = torch.from_numpy(points_arr).float()

    # 加载预训练模型
    state_dict = torch.load(segment_model_path)

    # 初始化分类器
    classifier = PointNetDenseCls(k=state_dict['conv4.weight'].size()[0])

    # 加载模型参数
    classifier.load_state_dict(state_dict)

    # 切换模型为评估模式
    classifier.eval()

    # 检查是否有可用的 GPU，如果没有则使用 CPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用设备: ", device)

    # 将点云数据转换为网络输入格式
    point = point.transpose(1, 0).contiguous()
    point = Variable(point.view(1, point.size()[0], point.size()[1]))

    # 将点云数据移动到指定设备（GPU/CPU）
    point = point.to(device)
    classifier = classifier.to(device)

    # 执行前向传播，获取预测结果
    pred, _, _ = classifier(point)

    # 获取预测类别
    pred_choice = pred.data.max(2)[1]
    print(pred_choice)

    # 获取颜色映射，用于显示点云的颜色
    cmap = plt.cm.get_cmap("hsv", 10)
    cmap = np.array([cmap(i) for i in range(10)])[:, :3]
    pred_color = cmap[pred_choice.cpu().numpy()[0], :]

    # 显示点云和预测结果
    showpoints(points_arr, pred_color, pred_color)

# Z. 模型预测

## 测试1

In [1]:
model_path = './seg/seg_model_Chair_0.pth'
ply_file = './test_datesets.lee/chair_0011.off'
segment_point_cloud(load_off(ply_file), model_path)

NameError: name 'segment_point_cloud' is not defined